In [5]:
# Import Required Modules
from flask import Flask, render_template
import pandas as pd
import json
import plotly
import plotly.express as px
import pandas_ta as pta
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import date
from datetime import datetime
from datetime import timedelta, date
from prophet import Prophet
import pandas as pd
import numpy as np
import yfinance as yf
import seaborn as sns
import mpld3
import matplotlib.pyplot as plt
import warnings

   
df = pd.read_csv("Resources/btcjoin.csv", parse_dates=['date'])
btc_df = yf.Ticker('BTC-USD').history(period='7y',interval='1d',actions=False).reset_index()
btc_df = btc_df.loc[(btc_df['Date'] > '2022-10-25')]
btc_df['Close']=btc_df['Close'].astype("float")
df['price']=df['price'].str.replace(',','')
df['price']=df['price'].astype("float")
btc_df = btc_df.rename(columns={"Close": "price", "Date":"date"})
btc_df['date'] = btc_df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
btc_df['date'] = pd.to_datetime(btc_df['date'])

df = pd.merge(df, btc_df, on=['date', 'price'], how='outer')
df = pd.merge(df, btc_df, on=['date', 'price'], how='outer')
df = df.rename(columns={"value": "wallets"})
df = df.drop(columns=['volume','change', 'low', 'high', 'open','Unnamed: 0', "wallets", "address", "mined"])
df['200D'] = df['price'].rolling(200).mean()
df['300D'] = df['price'].rolling(300).mean()
df['50D'] = df['price'].rolling(50).mean()
df['7D'] = df['price'].rolling(7).mean()
# df = df.dropna()
df['meanavge'] = (df['200D'] + df['300D'] + df['50D'] )/3
# df = df.drop(columns=['200D','300D', '50D'])
df['meanvalue'] = df["price"] - df["meanavge"]
df['status'] = df['meanvalue'].apply(lambda x: '1' if x > 0 else '0')
df['status']=df['status'].astype("object")
df['price-meanavge']=df['price'] - df['meanavge']
df['move%'] = (df['price-meanavge']/(df['price'] + df['meanavge']))
bins = [-.43, -.18, 0, .18, .43]
group_names = ["Severely Oversold","Neutral Oversold", "Neutral Overbought","Severely Overbought"]
df["Valuation"] = pd.cut(df["move%"], bins, labels=group_names)

c:\Users\chris\anaconda3\envs\python_data\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [6]:
df['daily_pct_change'] = df['price'].pct_change() * 100
df.tail(10)

,date,price,Open_x,High_x,Low_x,Volume_x,Open_y,High_y,Low_y,Volume_y,...,300D,50D,7D,meanavge,meanvalue,status,price-meanavge,move%,Valuation,daily_pct_change
4938,2024-04-30,60636.855469,63839.417969,64703.332031,59120.066406,3.784084e+10,63839.417969,64703.332031,59120.066406,3.784084e+10,...,42073.832240,66890.052187,63360.610491,52711.176359,7925.679110,1,7925.679110,0.069923,Neutral Overbought,-5.019125
4939,2024-05-01,58254.011719,60609.496094,60780.500000,56555.292969,4.843978e+10,60609.496094,60780.500000,56555.292969,4.843978e+10,...,42168.314486,66625.506641,62500.198103,52706.809099,5547.202619,1,5547.202619,0.049992,Neutral Overbought,-3.929695
4940,2024-05-02,59123.433594,58253.703125,59602.296875,56937.203125,3.271181e+10,58253.703125,59602.296875,56937.203125,3.271181e+10,...,42264.251712,66346.305313,61734.730469,52698.994034,6424.439559,1,6424.439559,0.057452,Neutral Overbought,1.492467
4941,2024-05-03,62889.835938,59122.300781,63320.503906,58848.312500,3.317202e+10,59122.300781,63320.503906,58848.312500,3.317202e+10,...,42372.909362,66176.170156,61611.089844,52735.785481,10154.050457,1,10154.050457,0.087818,Neutral Overbought,6.370405
4942,2024-05-04,63891.472656,62891.031250,64494.957031,62599.351562,2.062048e+10,62891.031250,64494.957031,62599.351562,2.062048e+10,...,42485.310156,66065.924141,61678.565848,52795.629948,11095.842708,1,11095.842708,0.095091,Neutral Overbought,1.592685
4943,2024-05-05,64031.132812,63892.453125,64610.890625,62955.304688,1.829616e+10,63892.453125,64610.890625,62955.304688,1.829616e+10,...,42597.365697,66040.244453,61809.694754,52883.926551,11147.206262,1,11147.206262,0.095344,Neutral Overbought,0.218590
4944,2024-05-06,63161.949219,64038.312500,65494.902344,62746.238281,2.869793e+10,64038.312500,65494.902344,62746.238281,2.869793e+10,...,42705.835690,65935.670938,61712.670201,52942.628947,10219.320271,1,10219.320271,0.088018,Neutral Overbought,-1.357439
4945,2024-05-07,62334.816406,63162.761719,64390.457031,62285.980469,2.593073e+10,63162.761719,64390.457031,62285.980469,2.593073e+10,...,42812.312923,65831.395391,61955.236049,52997.782622,9337.033785,1,9337.033785,0.080957,Neutral Overbought,-1.309543
4946,2024-05-08,61187.941406,62332.640625,62986.085938,60877.128906,2.608817e+10,62332.640625,62986.085938,60877.128906,2.608817e+10,...,42911.352565,65816.898750,62374.368862,53078.079504,8109.861902,1,8109.861902,0.070974,Neutral Overbought,-1.839863
4947,2024-05-09,62481.296875,61171.207031,62799.566406,60719.507812,2.565510e+10,61171.207031,62799.566406,60719.507812,2.565510e+10,...,43018.509993,65708.251250,62854.063616,53131.728481,9349.568394,1,9349.568394,0.080870,Neutral Overbought,2.113742


In [7]:
df = df[['date', 'price', 'daily_pct_change']]
df


,date,price,daily_pct_change
0,2010-09-25,0.100000,NaN
1,2010-09-26,0.100000,0.000000
2,2010-09-27,0.100000,0.000000
3,2010-09-28,0.100000,0.000000
4,2010-09-29,0.100000,0.000000
...,...,...,...
4943,2024-05-05,64031.132812,0.218590
4944,2024-05-06,63161.949219,-1.357439
4945,2024-05-07,62334.816406,-1.309543
4946,2024-05-08,61187.941406,-1.839863


In [21]:
# Convert the dates to datetime objects
date1 = pd.to_datetime('2012-11-28')
date2 = pd.to_datetime('2016-07-9')
date3 = pd.to_datetime('2020-05-11')
date4 = pd.to_datetime('2024-04-20')

# Create the cycle DataFrames
cycle1 = df[df['date'] <= date1]
cycle2 = df[(df['date'] > date1) & (df['date'] <= date2)]
cycle3 = df[(df['date'] > date2) & (df['date'] <= date3)]
cycle4 = df[(df['date'] > date3) & (df['date'] <= date4)]
cycle5 = df[df['date'] > date4]

cycle1 = cycle1.reset_index(drop=True)
cycle2 = cycle2.reset_index(drop=True)
cycle3 = cycle3.reset_index(drop=True)
cycle4 = cycle4.reset_index(drop=True)
cycle5 = cycle5.reset_index(drop=True)


cycle1['cumulative_pct_change'] = cycle1['daily_pct_change'].cumsum()
cycle2['cumulative_pct_change'] = cycle2['daily_pct_change'].cumsum()
cycle3['cumulative_pct_change'] = cycle3['daily_pct_change'].cumsum()
cycle4['cumulative_pct_change'] = cycle4['daily_pct_change'].cumsum()
cycle5['cumulative_pct_change'] = cycle5['daily_pct_change'].cumsum()



,date,price,daily_pct_change,cumulative_pct_change
0,2024-04-21,64926.644531,-0.104312,-0.104312
1,2024-04-22,66837.679688,2.943376,2.839064
2,2024-04-23,66407.273438,-0.643957,2.195107
3,2024-04-24,64276.898438,-3.208045,-1.012938
4,2024-04-25,64481.707031,0.318635,-0.694303
5,2024-04-26,63755.320312,-1.126500,-1.820804
6,2024-04-27,63419.140625,-0.527297,-2.348100
7,2024-04-28,63113.230469,-0.482363,-2.830463
8,2024-04-29,63841.121094,1.153309,-1.677154
9,2024-04-30,60636.855469,-5.019125,-6.696279


In [24]:
# Ensure all cycles are in the list
cycles = [cycle1, cycle2, cycle3, cycle4, cycle5]

# Create a new figure
fig = go.Figure()

# Plot each cycle
for i, cycle in enumerate(cycles, 1):
    fig.add_trace(go.Scatter(x=cycle.index, y=cycle['cumulative_pct_change'], mode='lines', name=f'Cycle {i}'))

# Set labels and title
fig.update_layout(
    title='Cycles',
    xaxis_title='Index',
    yaxis_title='Cumulative Pct Change'
)

# Show the plot
fig.show()
